In [1]:
import sys

sys.path.append("../")
sys.path.append("../..")

import numpy as np
import qecstruct as qc
import qecsim.paulitools as pt
import matplotlib.pyplot as plt
from matplotlib import colormaps
from matplotlib.colors import LogNorm, Normalize
from matplotlib.ticker import FuncFormatter, FormatStrFormatter
from tqdm import tqdm
from scipy.stats import sem

from mdopt.mps.utils import create_custom_product_state
from mdopt.contractor.contractor import mps_mpo_contract
from mdopt.optimiser.utils import (
    SWAP,
    COPY_LEFT,
    XOR_BULK,
    XOR_LEFT,
    XOR_RIGHT,
)

from examples.decoding.decoding import (
    css_code_checks,
    css_code_logicals,
    css_code_logicals_sites,
    css_code_constraint_sites,
    apply_constraints,
    apply_bitflip_bias,
    apply_depolarising_bias,
    decode_custom,
    pauli_to_mps,
    decode_css,
    css_code_stabilisers,
    multiply_pauli_strings,
    map_distribution_to_pauli,
    generate_pauli_error_string,
)
from examples.decoding.visualisation import plot_parity_check_mpo

In [2]:
for NUM_BITS in [4, 8, 16]:
    CHECK_DEGREE, BIT_DEGREE = 4, 3
    NUM_CHECKS = int(NUM_BITS * BIT_DEGREE / CHECK_DEGREE)
    if NUM_BITS / NUM_CHECKS != CHECK_DEGREE / BIT_DEGREE:
        raise ValueError("The Tanner graph of the code must be bipartite.")
    SEED = 123
    ERROR_RATE = 0.06
    classical_code = qc.random_regular_code(NUM_BITS, NUM_CHECKS, BIT_DEGREE, CHECK_DEGREE, qc.Rng(SEED))
    qhgp_code = qc.hypergraph_product(classical_code, classical_code)
    CODE_PARAMETERS = (
        len(css_code_stabilisers(qhgp_code)[0][0]),
        len(css_code_stabilisers(qhgp_code)[0][0]) - qhgp_code.num_x_stabs() - qhgp_code.num_z_stabs(),
    )
    print(f"Code parameters: {CODE_PARAMETERS}")
    seed_seq = np.random.SeedSequence(SEED)
    rng = np.random.default_rng(seed_seq.spawn(1)[0])
    error = generate_pauli_error_string(len(qhgp_code), ERROR_RATE, rng=rng, error_model="Bitflip")
    state, success = decode_css(
                code=qhgp_code,
                error=error,
                chi_max=1,
                multiply_by_stabiliser=False,
                bias_type="Bitflip",
                bias_prob=0.1,
                tolerance=1e-1,
                cut=1e-1,
                renormalise=True,
                silent=False,
                contraction_strategy="Optimised",
                optimiser="Dephasing DMRG",
                num_runs=1,
            )


Code parameters: (25, 1)
Starting the decoding.
The total number of sites: 76.
Applying bitflip bias.
Applying X logicals' constraints.


100%|██████████| 13/13 [00:00<00:00, 121.09it/s]

Applying Z logicals' constraints.



100%|██████████| 13/13 [00:00<00:00, 221.73it/s]

Applying the X checks' constraints.


...

Applying the X checks' constraints.



100%|██████████| 192/192 [00:16<00:00, 11.65it/s]

Marginalising the error MPS.
The number of logical sites: 32.
Running the Dephasing DMRG engine.



100%|██████████| 1/1 [00:00<00:00,  9.54it/s]

Dephasing DMRG run completed with overlap: 0.000000


In [3]:
import numpy as np
from scipy.linalg import null_space, lstsq
from tqdm import tqdm

def compute_css_distance(css_code):
    """
    Computes the distance of a CSS code given its X and Z parity check matrices
    in sparse list-of-lists format.

    Parameters
    ----------
    css_code : object
        Must have parity check matrices accessible via `css_code_checks(css_code)`,
        returning (Hx_rows, Hz_rows), each a list of lists of qubit indices.

    Returns
    -------
    distance : int
        The code distance (minimum weight of a nontrivial logical operator).
    """
    Hx_rows, Hz_rows = css_code_checks(css_code)

    # Correctly infer the number of physical qubits from check matrix support
    all_indices = [i for row in Hx_rows + Hz_rows for i in row]
    n_qubits = max(all_indices) + 1

    def rows_to_matrix(rows):
        mat = np.zeros((len(rows), n_qubits), dtype=np.uint8)
        for i, row in enumerate(rows):
            mat[i, row] = 1
        return mat

    def binary_nullspace(H):
        H = H % 2
        ns = null_space(H.astype(float))
        ns_bin = (np.round(ns) % 2).astype(np.uint8).T
        return ns_bin[np.any(ns_bin, axis=1)]

    def min_logical_weight(null_basis, other_H):
        min_wt = np.inf
        for v in tqdm(null_basis):
            res, _, _, _ = lstsq(other_H.T.astype(float), v.astype(float))
            lhs = np.round(other_H.T @ res) % 2
            if not np.allclose(lhs, v % 2, atol=1e-4):
                wt = np.sum(v)
                if 0 < wt < min_wt:
                    min_wt = wt
        return min_wt

    Hx = rows_to_matrix(Hx_rows)
    Hz = rows_to_matrix(Hz_rows)

    Cx_perp = binary_nullspace(Hx)
    Cz_perp = binary_nullspace(Hz)

    dx = min_logical_weight(Cz_perp, Hx)
    dz = min_logical_weight(Cx_perp, Hz)

    return int(min(dx, dz))


In [4]:
compute_css_distance(qhgp_code)

100%|██████████| 519/519 [03:22<00:00,  2.56it/s]


1